### 입력 데이터:

* 다운로드: input.zip
* 설명: 영화DB사이트 (MovieLens)에서 수집된 데이터로서 u.data에는 사용자가 영화에 매긴 평점이 담겨있고 u.item에는 영화마다 제목, 카테고리 등의 정보가 담겨 있다.


#### 입력파일 1 - u.data


* 라인마다 사용자의 영화평점이 기록되어 있으며 4개의 속성 "user id \t item id \t rating \t timestamp"가 탭으로 구분되어 작성되어 있다. (즉, split("\t") 로 자를 수 있음)
* 모두 integer값이다.
* 한 사람이 여러개의 영화에 평점을 매길 수 있으므로 같은 user id로 시작하는 라인이 복수번 나타날수 있다.
* item id는 아래 u.item파일에 있는 영화 아이디 (movie id)를 가리킨다 (즉, foreign key)
* rating은 1에서 5까지 범위이다.


#### 입력파일 2 - u.item


* 라인마다 영화 정보가 기록되어 있으며 "movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western" 의 속성 들이 파이프라인(|)으로 구분되어 작성되어 있다.
* 그러나 아래 문제를 보면 맨 앞의 두개 속성, movie id와 movie title 만 사용한다.
* movie id는 이 파일 안에서 한번만 나타난다 (즉, primary key)


### 문제 1.
* 5점을 받은 영화 중에서 알파벳 'T'로 시작하는 영화의 개수는?
* 'T'는 대문자
* 주의: 5점을 받은 영화중 알파벳 'T'로 시작하는 영화를 중복해서 세지 않도록!
* 제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.

### 문제 2.
* 사용자들에게 1점도 받고 5점도 받은 적이 있는 영화의 제목의 길이 중 가장 긴 영화제목의 길이는 [공백 1개]이다.
* 동일 영화가 어떤 사용자에게는 1점, 어떤 사용자에게는 5점을 받은 경우를 말함
* 힌트: cogroup을 쓸 수도 있고 groupByKey를 쓸 수도 있음.
* 제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext("local", "transformation 3")

In [2]:
u_data = sc.textFile("u.data")
u_data = u_data.map(lambda x: tuple(x.split("\t")[1:3]))

In [3]:
u_data.take(10)

[('242', '3'),
 ('302', '3'),
 ('377', '1'),
 ('51', '2'),
 ('346', '1'),
 ('474', '4'),
 ('265', '2'),
 ('465', '5'),
 ('451', '3'),
 ('86', '3')]

In [4]:
u_item = sc.textFile("u.item")
u_item = u_item.map(lambda x: tuple(x.split("|")[:2]))

In [5]:
u_item.take(10)

[('1', 'Toy Story (1995)'),
 ('2', 'GoldenEye (1995)'),
 ('3', 'Four Rooms (1995)'),
 ('4', 'Get Shorty (1995)'),
 ('5', 'Copycat (1995)'),
 ('6', 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'),
 ('7', 'Twelve Monkeys (1995)'),
 ('8', 'Babe (1995)'),
 ('9', 'Dead Man Walking (1995)'),
 ('10', 'Richard III (1995)')]

In [6]:
u_item.count() == u_item.map(lambda x: x[0]).distinct().count()

True

## 5점을 받은 영화 중에서 알파벳 'T'로 시작하는 영화의 개수 

In [7]:
u_item.filter(lambda x: x[1][0] == 'T').take(5)

[('1', 'Toy Story (1995)'),
 ('7', 'Twelve Monkeys (1995)'),
 ('23', 'Taxi Driver (1976)'),
 ('40', 'To Wong Foo, Thanks for Everything! Julie Newmar (1995)'),
 ('59', 'Three Colors: Red (1994)')]

In [8]:
u_data.filter(lambda x: x[1] == '5').take(5)

[('465', '5'), ('1014', '5'), ('222', '5'), ('387', '5'), ('95', '5')]

In [9]:
u_data.filter(lambda x: x[1] == '5').join(u_item.filter(lambda x: x[1][0] == 'T')).distinct().count()

75

## 1점도 받고 5점도 받은 영화의 제목의 길이중 가장 긴 영화제목의 길이는?

In [10]:
u_data \
  .cogroup(u_item) \
  .mapValues(lambda t: (list(t[0]), list(t[1]))) \
  .filter(lambda t: ('5' in t[1][0]) and ('1' in t[1][0])) \
  .map(lambda t: len(t[1][1][0])) \
  .max()

81